In [3]:
%pip install -q -U openai psycopg2-binary pgvector

Note: you may need to restart the kernel to use updated packages.


docker run -d --name pgvector -p 5432:5432 -e POSTGRES_PASSWORD=YOUR_PASSWORD -e POSTGRES_USER=pgadmin -e POSTGRES_DB=pgvector pgvector/pgvector:pg16

In [43]:
from psycopg2 import pool, extras, sql, Error
from psycopg2.extras import execute_values
from pgvector.psycopg2 import register_vector
import os
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version=os.getenv("OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)


### Connect to Postgres (PG)

In [39]:
conn_string = "host={0} user={1} dbname={2} password={3} sslmode={4}".format('127.0.0.1', 'pgadmin', 'pgvector', 'P@ssw0rd', 'disable')
pg_pool = psycopg2.pool.SimpleConnectionPool(1, 20, conn_string)
if (pg_pool):
    print("Connection pool created successfully")

# Use getconn() to get a connection from the connection pool
connection = pg_pool.getconn()


Connection pool created successfully


In order to use pgvector, we need to first create the vector extension as described in this link and shown below.

In [41]:
try:
    with connection.cursor() as cursor:
        # Check if the extension already exists
        extension_query = "SELECT * FROM pg_extension WHERE extname = 'vector';"
        cursor.execute(extension_query)
        extension_exists = cursor.fetchone()

        if not extension_exists:
            # Extension does not exist, create it
            create_extension_query = "CREATE EXTENSION vector;"
            cursor.execute(create_extension_query)
            connection.commit()
        else:
            # Extension already exists, pass through
            print("Extension already exists")

        connection.commit()
except (Exception, Error) as e:
    print(f"Error: {e}")
    connection.rollback()   

Extension already exists


In the following we will list the existing extensions for your db. Please make sure ['VECTOR'] IS listed.

In [42]:
# Define the SHOW EXTENSIONS query
show_extensions_query = "select extname from pg_extension;"

# Execute the CREATE TABLE query
try:
    with connection.cursor() as cursor:
        cursor.execute(show_extensions_query)
        # Fetch and print the results
        results = cursor.fetchall()
        for row in results:
            print(row)        
        connection.commit()
except (Exception, Error) as e:
    print(f"Error: {e}")
    connection.rollback()

('plpgsql',)
('vector',)


In [16]:
import os
import json
import time

def generate_embeddings(text):
    '''
    Generate embeddings from string of text.
    This will be used to vectorize data and user input for interactions with Azure OpenAI.
    '''
    response = client.embeddings.create(input=text,  model="text-embedding-ada-002")
    embeddings =response.model_dump()
    time.sleep(0.5) 
    return embeddings['data'][0]['embedding']

def generate_document_embeddings(input_json_path, output_json_path):
    # Read the input JSON file
    with open(input_json_path, 'r', encoding='utf-8') as file:
        input_data = json.load(file)

    titles = [item['title'] for item in input_data]
    content = [item['content'] for item in input_data]
    
    # Generate embeddings for titles
    title_response = client.embeddings.create(input=titles, model="text-embedding-ada-002")
    title_embeddings = [item.embedding for item in title_response.data]
    
    # Generate embeddings for content
    content_response = client.embeddings.create(input=content, model="text-embedding-ada-002")
    content_embeddings = [item.embedding for item in content_response.data]

    # Assign embeddings to the original data
    for i, item in enumerate(input_data):
        item['titleVector'] = title_embeddings[i]
        item['contentVector'] = content_embeddings[i]

    # Ensure the output directory exists
    output_directory = os.path.dirname(output_json_path)
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    
    # Write the modified data to the output JSON file
    with open(output_json_path, "w") as f:
        json.dump(input_data, f)


# Generate and load the documents with embeddings from the output file
output_path = os.path.join('output', 'docVectors.json')
if not os.path.exists(output_path):
    generate_document_embeddings(os.path.join('..', 'data', 'text-sample.json'), output_path)
    
with open(output_path, 'r') as file:  
    documents = json.load(file)

In [44]:
import pandas as pd
import numpy as np
df = pd.DataFrame(documents, columns=['id', 'title', 'content', 'category', 'titleVector', 'contentVector'])
data_list = [(int(row['id']), row['title'], row['content'], row['category'], np.array(row['titleVector']), np.array(row['contentVector'])) for index, row in df.iterrows()]


Load the data and vectors to a table in the database
In this section, we will load the data into a pandas dataframe, use select columns, and create vector embedding using azure open ai.

In [47]:
# Register 'pgvector' type
register_vector(connection)

# # Drop previous table of same name if one exists
"""
drop_table_query = "DROP TABLE IF EXISTS vs;"
try:
    with connection.cursor() as cursor:
        cursor.execute(drop_table_query)
        print("Finished dropping table (if existed)")
        connection.commit()
except (Exception, Error) as e:
    print(f"Error: {e}")
    connection.rollback()
"""

# Execute the query to check if the table exists
table_exists_query = "SELECT EXISTS (SELECT 1 FROM information_schema.tables WHERE table_name = 'vs');"
try:
    with connection.cursor() as cursor:
        cursor.execute(table_exists_query)
        # Fetch and print the results
        exists = cursor.fetchone()[0]       
        connection.commit()
except (Exception, Error) as e:
    print(f"Error: {e}")
    connection.rollback()
    raise

if exists:
    print(f"The table vs exists in the database.")
    print("You may drop previous table if you want to re-insert data.")
else:
    print(f"The table vs does not exist in the database. Creating it now and inserting data ...")
    # Define the CREATE TABLE query
    create_table_query = f"""
    CREATE TABLE IF NOT EXISTS vs (
        Id INTEGER PRIMARY KEY,
        Title TEXT,
        Content TEXT,
        Category TEXT,
        TitleVector VECTOR(1536),
        ContentVector VECTOR(1536)
    );
    """

    # Execute the CREATE TABLE query
    try:
        with connection.cursor() as cursor:
            cursor.execute(create_table_query)
            connection.commit()
            print(f"Table vs created successfully!")
    except (Exception, Error) as e:
        print(f"Error creating table vs: {e}")
        connection.rollback()     

    # Batch insert data and embeddings into PostgreSQL database
    try:
        with connection.cursor() as cursor:
            execute_values(cursor, "INSERT INTO vs (Id, Title, Content, Category, TitleVector, ContentVector) VALUES %s", data_list)
            connection.commit()
            print(f"Data inserted to table vs successfully!")
    except (Exception, Error) as e:
        print(f"Error inserting data to vs: {e}")
        connection.rollback()           
    


The table vs exists in the database.
You may drop previous table if you want to re-insert data.


User Asks a Question
In this step, the code will convert the user's question to an embedding and then retieve the top K document chunks based on the users' question using the similarity. Please note that other similarity metrics can also be used.

In [54]:
question = "What is Azure App Service?"
retrieve_k = 3 # for retrieving the top k reviews from the database
# Generate embeddings for the question and retrieve the top k document chunks
question_embedding = generate_embeddings(question)
select_query = f"SELECT Id, Title, Content, Category, ContentVector <-> %s::vector as distance FROM vs ORDER BY ContentVector <-> %s::vector LIMIT {retrieve_k}"

try:
    with connection.cursor() as cursor:
        cursor.execute(select_query, (np.array(question_embedding),np.array(question_embedding),))
        results = cursor.fetchall()
        connection.commit()
except (Exception, Error) as e:
    print(f"Error: {e}")
    connection.rollback()

In [55]:
results

[(1,
  'Azure App Service',
  'Azure App Service is a fully managed platform for building, deploying, and scaling web apps. You can host web apps, mobile app backends, and RESTful APIs. It supports a variety of programming languages and frameworks, such as .NET, Java, Node.js, Python, and PHP. The service offers built-in auto-scaling and load balancing capabilities. It also provides integration with other Azure services, such as Azure DevOps, GitHub, and Bitbucket.',
  'Web',
  0.4546572331058624),
 (86,
  'Azure App Configuration',
  'Azure App Configuration is a fully managed configuration service that enables you to centrally manage and distribute your application settings and feature flags. It provides features like key-value storage, versioning, and access control. App Configuration supports various platforms, such as .NET, Java, Node.js, and Python. You can use Azure App Configuration to build and deploy your applications, ensure the consistency of your settings, and improve your